# Proyecto IDS
## Integrantes:
- Vicente Alarcón
- Matías Fontecilla

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

In [2]:
path_file = 'games.csv'
columns = ['AppID', 'Name', 'Release date', 'Estimated owners', 'Price', 'Metacritic score', 'Positive', 'Negative', 'Categories', 'Genres', 'Developers', 'Publishers', 'Median playtime forever', 'Average playtime forever', 'Recommendations', 'User score', 'Supported languages']
df = pd.read_csv(path_file, usecols=columns)
df = df[df['Metacritic score'] > 0]

df['Steam score'] = ((df['Positive'] / (df['Positive'] + df['Negative'])) * 100).round()
df['Steam score'] = df['Steam score'].replace([np.inf, -np.inf], np.nan).fillna(0).astype(int)


df['Estimated owners'] = df['Estimated owners'].apply(lambda x: (int(x.split('-')[0]) + int(x.split('-')[1])) / 2)


df['Estimated earnings'] = df['Estimated owners'] * df['Price']

df = df.loc[df['Positive'] + df['Negative'] > 500]

numeric_columns = ['Estimated owners', 'Price', 'Metacritic score', 'Positive', 'Negative', 'Median playtime forever', 'Average playtime forever', 'Recommendations', 'User score']
for col in numeric_columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    filter = (df[col] >= Q1 - 1.5 * IQR) & (df[col] <= Q3 + 1.5 * IQR)
    df = df.loc[filter] 

Q1 = df['Median playtime forever'].quantile(0.25)
Q3 = df['Median playtime forever'].quantile(0.75)
IQR = Q3 - Q1
filter = (df['Median playtime forever'] >= Q1 - 1.5 * IQR) & (df['Median playtime forever'] <= Q3 + 1.5 *IQR)
df = df.loc[filter]

df = df.sort_values(by=['Steam score', 'Metacritic score'], ascending=False)
df.to_csv('filtered_games.csv', index=False)

In [ ]:
# Leer el archivo CSV
df = pd.read_csv('filtered_games.csv')

# Seleccionar las columnas relevantes
df = df[['AppID', 'Name', 'Metacritic score', 'Steam score', 'Estimated earnings']]

# Crear los intervalos
bins = range(0, 101, 10)

# Dividir los datos en intervalos
df['Intervalo Metacritic'] = pd.cut(df['Metacritic score'], bins)

# Dividir los datos en intervalos
df['Intervalo Steam'] = pd.cut(df['Steam score'], bins)

# Calcular el promedio para cada intervalo
promedio_por_intervalo_metacritic = df.groupby('Intervalo Metacritic', observed=True)['Metacritic score'].mean()

# Calcular el promedio para cada intervalo
promedio_por_intervalo_steam = df.groupby('Intervalo Steam', observed=True)['Steam score'].mean()

# Crear un DataFrame con los promedios por intervalo
df_promedios = pd.DataFrame({
    'Metacritic': promedio_por_intervalo_metacritic,
    'Steam': promedio_por_intervalo_steam,
})

# Graficar los promedios en un solo gráfico de barras
df_promedios.plot.bar(color=['blue', 'green'], figsize=(8, 6))
plt.title('Promedio por intervalo para Metacritic, Steam')
plt.xlabel('Intervalo')
plt.ylabel('Promedio')
plt.show()

In [ ]:
# Leer el archivo CSV
df = pd.read_csv('filtered_games.csv')

# Seleccionar las columnas relevantes
df = df[['AppID', 'Name', 'Metacritic score', 'Steam score', 'Estimated earnings']]

# Crear los intervalos
bins = range(0, 101, 10)

# Dividir los datos en intervalos
df['Intervalo Metacritic'] = pd.cut(df['Metacritic score'], bins)
df['Intervalo Steam'] = pd.cut(df['Steam score'], bins)

# Calcular el promedio de ganancias para cada intervalo de Metacritic
promedio_ganancias_por_intervalo_metacritic = df.groupby('Intervalo Metacritic', observed=True)['Estimated earnings'].mean()

# Calcular el promedio de ganancias para cada intervalo de Steam
promedio_ganancias_por_intervalo_steam = df.groupby('Intervalo Steam', observed=True)['Estimated earnings'].mean()

# Crear un DataFrame con los promedios de ganancias por intervalo
df_promedios = pd.DataFrame({
    'Metacritic': promedio_ganancias_por_intervalo_metacritic,
    'Steam': promedio_ganancias_por_intervalo_steam,
})

# Graficar los promedios de ganancias en un solo gráfico de barras
ax = df_promedios.plot.bar(color=['blue', 'green'], figsize=(8, 6))
plt.title('Promedio de ganancias por intervalo de puntaje para Metacritic y Steam')
plt.xlabel('Intervalo')
plt.ylabel('Promedio de ganancias')

# Formatear el eje Y para mostrar el número completo
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

# print(promedio_ganancias_por_intervalo_metacritic)
# print(promedio_ganancias_por_intervalo_steam)

plt.show()

In [ ]:
df = pd.read_csv('filtered_games.csv')
df = df[['AppID', 'Name', 'Metacritic score', 'Steam score']]
# Calcular la diferencia entre las puntuaciones
df['Diferencia'] = df['Steam score'] - df['Metacritic score']

# Calcular las medias
metacritic_mean = df['Metacritic score'].mean()
steam_mean = df['Steam score'].mean()

# Calcular la diferencia
diferencia = steam_mean - metacritic_mean

# Crear un gráfico de puntos
plt.figure(figsize=(12, 6))

# Dibujar los puntos
plt.scatter(df.index, df['Diferencia'], color='purple', alpha=0.7)

# Añadir líneas horizontales para resaltar la diferencia promedio
plt.axhline(y=0, color='black', linestyle='--', linewidth=2, label='Diferencia Promedio')
plt.axhline(y=7, color='red', linestyle='--', linewidth=2, label='Diferencia de 7 puntos')

# Añadir etiquetas y título
plt.xlabel('Juegos')
plt.ylabel('Diferencia de Puntuación (Steam - Metacritic)')
plt.title('Diferencia de Puntuación entre Steam y Metacritic para Cada Juego')
plt.legend()

# Mostrar el gráfico
plt.show()

In [4]:
# Leer el archivo CSV
df = pd.read_csv('filtered_games.csv')

# Seleccionar las columnas relevantes
df = df[['Metacritic score', 'Steam score', 'Positive', 'Negative', 'Estimated earnings']]

# Eliminar filas con valores nulos
df = df.dropna()

# Definir las variables independientes (X) y la variable dependiente (y)
X = df.drop('Estimated earnings', axis=1)
y = df['Estimated earnings']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
rf_predictions = rf_model.predict(X_test)

# Evaluar el rendimiento del modelo de Random Forest
rf_mse = mean_squared_error(y_test, rf_predictions)
rf_r2 = r2_score(y_test, rf_predictions)

# Obtener la importancia de las características
feature_importance = rf_model.feature_importances_
# Asociar importancia con nombres de características
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importance})
# Ordenar por importancia descendente
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print(feature_importance_df)

print(f'Mean Squared Error (Random Forest): {rf_mse}')
print(f'R2 Score (Random Forest): {rf_r2}')

            Feature  Importance
2          Positive    0.431501
0  Metacritic score    0.249317
3          Negative    0.209515
1       Steam score    0.109666
Mean Squared Error (Random Forest): 14927319431878.91
R2 Score (Random Forest): 0.18185657288041845


In [6]:
df = pd.read_csv('filtered_games.csv')

df = df[['Metacritic score', 'Steam score', 'Estimated earnings','Positive','Negative'
,'Categories', 'Developers'
]]
df = df.dropna()

df = pd.get_dummies(df, columns=['Categories', 'Developers'])

X = df.drop('Estimated earnings', axis=1)
y = df['Estimated earnings']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = [
    ('Linear Regression', LinearRegression()),
    ('Ridge', Ridge()),
    ('Lasso', Lasso()),
    ('Decision Tree Regressor', DecisionTreeRegressor()),
    ('Random Forest Regressor', RandomForestRegressor()),
    ('Gradient Boosting Regressor', GradientBoostingRegressor()),
    ('MLP Regressor', MLPRegressor(max_iter=10000))
]

for name, model in models:
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print("---------------------------------------------------")
    print(f"Precisión del modelo {name}: {score}")

---------------------------------------------------
Precisión del modelo Linear Regression: -0.7429051748767312
---------------------------------------------------
Precisión del modelo Ridge: 0.20613000322690267


c:\Users\Vichito\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.075e+14, tolerance: 1.769e+12
  model = cd_fast.enet_coordinate_descent(


---------------------------------------------------
Precisión del modelo Lasso: -0.6977979425351917
---------------------------------------------------
Precisión del modelo Decision Tree Regressor: 0.024009653624088134
---------------------------------------------------
Precisión del modelo Random Forest Regressor: 0.2164294232634938
---------------------------------------------------
Precisión del modelo Gradient Boosting Regressor: 0.21957822308382713
---------------------------------------------------
Precisión del modelo MLP Regressor: 0.2472980564403675
